# Projeto 3: Classificação binária brest cancer com tuning dos parâmetros

## Etapa 1: Importação das bibliotecas

In [ ]:
!pip install skorch

     |████████████████████████████████| 155 kB 11.6 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import torch.nn as nn
from skorch import NeuralNetBinaryClassifier 
import torch
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV
torch.__version__

'1.10.0+cu111'

## Etapa 2: Base de dados

In [ ]:
np.random.seed(123)
torch.manual_seed(123)

In [ ]:
previsores = pd.read_csv('/content/entradas_breast.csv')
classe = pd.read_csv('/content/saidas_breast.csv')

In [ ]:
previsores.shape

(569, 30)

In [ ]:
previsores.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave_points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1095.0000,0.9053,8589.0,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3398.0,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,186.0000,275.0000,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4585.0,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,243.0000,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1156.0000,3445.0,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,173.00000
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,198.0000,0.10430,0.1809,0.05883,0.7572,0.7813,5438.0,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,205.0000,0.4000,0.1625,0.2364,0.07678


In [ ]:
classe.shape

(569, 1)

In [ ]:
# Precisamos converter de pandas para numpy que é o formato aceito pelo pytorch

previsores = np.array(previsores, dtype='float32')
classe = np.array(classe, dtype='float32').squeeze(1)

In [ ]:
classe

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1.,
       1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1.,
       1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1.,
       0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1.,
       1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1.,
       1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1.,
       1., 1., 0., 1., 1.

## Etapa 3: Classe para estrutura da rede neural

In [ ]:
# Aqui instanciamos a rede neural
# É necessário invocar o nn.Module 
class classificador_torch(nn.Module):
  # Vamos passar listas de parâmetros e o gridSearch escolherá o melhor resultado
  # activation será a lista de funções de ativação q usaremos
  # neurons qtd de neurônios
  # Initializer é o inicializador dos pesos
  def __init__(self, activation, neurons, initializer):
    super().__init__()
    # 30 -> 16 -> 16 -> 1
    self.dense0 = nn.Linear(30, neurons)
    initializer(self.dense0.weight)
    self.activation0 = activation
    self.dense1 = nn.Linear(neurons, neurons)
    initializer(self.dense1.weight)
    self.activation1 = activation
    self.dense2 = nn.Linear(neurons, 1)
    initializer(self.dense2.weight)
    self.output = nn.Sigmoid()

  def forward(self, X):
    X = self.dense0(X)
    X = self.activation0(X)
    X = self.dense1(X)
    X = self.activation1(X)
    X = self.dense2(X)
    X = self.output(X)
    return X

## Etapa 4: Skorch

In [ ]:
# Vamos instanciar a rede em que no module passaremos nossa função
classificador_sklearn = NeuralNetBinaryClassifier(module = classificador_torch, lr = 0.001, optimizer__weight_decay = 0.0001, train_split = False)

## Etapa 5: Tuning dos parâmetros

In [ ]:
# Precisamos instanciar uma variável no formato de dicionário com os campos que queremos fazer o gridSearch

params = {'batch_size': [10,30],
          'max_epochs':[50,100],
          'optimizer': [torch.optim.Adam, torch.optim.SGD], # SGD é a descida estocástica clássica
          'criterion': [torch.nn.BCELoss, torch.nn.HingeEmbeddingLoss],
          'module__activation': [F.relu, F.tanh],
          'module__neurons': [8,16],
          'module__initializer': [torch.nn.init.uniform_, torch.nn.init.normal_]
    


}

In [ ]:
params = {'batch_size': [10],
          'max_epochs': [100],
          'optimizer': [torch.optim.Adam, torch.optim.SGD],
          'criterion': [torch.nn.BCELoss], #, torch.nn.HingeEmbeddingLoss],
          'module__activation': [F.relu, F.tanh],
          'module__neurons': [8, 16], 
          'module__initializer': [torch.nn.init.uniform]} # _, torch.nn.init.normal_]}

A melhor estratégia é testar cada um dos parâmetros um por um para saber a melhor combinação de valores. Ou seja, testa primeiro o melhor batch_size, depois o melhor max_epochs...

In [ ]:
params

{'batch_size': [10, 30],
 'criterion': [torch.nn.modules.loss.BCELoss,
  torch.nn.modules.loss.HingeEmbeddingLoss],
 'max_epochs': [50, 100],
 'module__activation': [<function torch.nn.functional.relu>,
  <function torch.nn.functional.tanh>],
 'module__initializer': [<function torch.nn.init.uniform_>,
  <function torch.nn.init.normal_>],
 'module__neurons': [8, 16],
 'optimizer': [torch.optim.adam.Adam, torch.optim.sgd.SGD]}

In [ ]:
# Vamos instanciar o objeto gridSearch
grid_search = GridSearchCV(estimator=classificador_sklearn, param_grid = params, scoring = 'accuracy', cv = 10)

# Agora damos o fit
grid_search = grid_search.fit(previsores, classe)

  epoch    train_loss     dur
-------  ------------  ------
      1       37.1094  0.2908
      2       37.1094  0.0715
      3       37.1094  0.0699
      4       37.1094  0.0695
      5       37.1094  0.0697
      6       37.1094  0.0832
      7       37.1094  0.0696
      8       37.1094  0.0751
      9       37.1094  0.0711
     10       37.1094  0.0706
     11       37.1094  0.0699
     12       37.1094  0.0714
     13       37.1094  0.0759
     14       37.1094  0.0701
     15       37.1094  0.0699
     16       37.1094  0.0707
     17       37.1094  0.0751
     18       37.1094  0.0714
     19       37.1094  0.0739
     20       37.1094  0.0705
     21       37.1094  0.0706
     22       13.5593  0.0769
     23        0.6080  0.0731
     24        0.5856  0.0728
     25        0.5634  0.0716
     26        0.5449  0.0778
     27        0.5280  0.0764
     28        0.5103  0.0651
     29        0.4914  0.0741
     30        0.4733  0.0706
     31        0.4508  0.0719
     32   

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:775: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/skorch/classifier.py", line 357, in predict
    return (y_proba[:, 1] > self.threshold).astype('uint8')
IndexError: too many i

      3       37.1094  0.0720
      4       37.1094  0.0774
      5       37.1094  0.0697
      6       37.1094  0.0699
      7       37.1094  0.0718
      8       37.1094  0.0771
      9       37.1094  0.0736
     10       37.1094  0.0728
     11       37.1094  0.0716
     12       37.1094  0.0723
     13       37.1094  0.0718
     14       37.1094  0.0722
     15       37.1094  0.0713
     16       37.1094  0.0861
     17       37.1094  0.0792
     18       37.1094  0.0692
     19       37.1094  0.0699
     20       37.1094  0.0697
     21       23.9451  0.0717
     22        0.5928  0.0701
     23        0.5412  0.0759
     24        0.5097  0.0762
     25        0.5010  0.0725
     26        0.4902  0.0686
     27        0.4865  0.0705
     28        0.4632  0.0711
     29        0.4465  0.0716
     30        0.4157  0.0709
     31        0.3892  0.0766
     32        0.3969  0.0691
     33        0.3617  0.0749
     34        0.3490  0.0694
     35        0.3474  0.0700
     36   

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:775: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/skorch/classifier.py", line 357, in predict
    return (y_proba[:, 1] > self.threshold).astype('uint8')
IndexError: too many i

      3       37.3047  0.0723
      4       37.3047  0.0671
      5       37.3047  0.0697
      6       37.3047  0.0681
      7       37.3047  0.0682
      8       37.3047  0.0740
      9       37.3047  0.0682
     10       37.3047  0.0687
     11       37.3047  0.0676
     12       37.3047  0.0681
     13       37.3047  0.0692
     14       37.3047  0.0659
     15       37.3047  0.0738
     16       37.3047  0.0687
     17       37.3047  0.0758
     18       37.3047  0.0679
     19       37.3047  0.0715
     20       12.3071  0.0701
     21        0.6852  0.0672
     22        0.6808  0.0689
     23        0.6776  0.0793
     24        0.6751  0.0705
     25        0.6732  0.0692
     26        0.6716  0.0718
     27        0.6703  0.0780
     28        0.6692  0.0718
     29        0.6682  0.0707
     30        0.6674  0.0715
     31        0.6667  0.0693
     32        0.6661  0.0725
     33        0.6655  0.0684
     34        0.6650  0.0688
     35        0.6646  0.0749
     36   

...

      3        1.0410  0.0681
      4        0.9994  0.0765
      5        0.9604  0.0653
      6        0.9244  0.0645
      7        0.8912  0.0702
      8        0.8610  0.0655
      9        0.8338  0.0644
     10        0.8093  0.0640
     11        0.7877  0.0666
     12        0.7686  0.0754
     13        0.7519  0.0642
     14        0.7375  0.0668
     15        0.7251  0.0645
     16        0.7145  0.0627
     17        0.7055  0.0662
     18        0.6979  0.0707
     19        0.6915  0.0644
     20        0.6861  0.0735
     21        0.6817  0.0641
     22        0.6780  0.0637
     23        0.6749  0.0621
     24        0.6724  0.0653
     25        0.6703  0.0714
     26        0.6686  0.0628
     27        0.6672  0.0658
     28        0.6660  0.0670
     29        0.6651  0.0670
     30        0.6643  0.0646
     31        0.6637  0.0624
     32        0.6632  0.0626
     33        0.6628  0.0759
     34        0.6624  0.0644
     35        0.6621  0.0651
     36   

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:775: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/skorch/classifier.py", line 357, in predict
    return (y_proba[:, 1] > self.threshold).astype('uint8')
IndexError: too many i

      3        1.0337  0.0667
      4        0.9923  0.0739
      5        0.9537  0.0660
      6        0.9180  0.0672
      7        0.8852  0.0719
      8        0.8555  0.0643
      9        0.8286  0.0673
     10        0.8047  0.0615
     11        0.7834  0.0633
     12        0.7648  0.0704
     13        0.7486  0.0636
     14        0.7346  0.0644
     15        0.7225  0.0699
     16        0.7123  0.0666
     17        0.7036  0.0623
     18        0.6963  0.0626
     19        0.6901  0.0643
     20        0.6850  0.0612
     21        0.6807  0.0629
     22        0.6772  0.0737
     23        0.6742  0.0624
     24        0.6718  0.0685
     25        0.6699  0.0705
     26        0.6682  0.0691
     27        0.6669  0.0679
     28        0.6658  0.0633
     29        0.6649  0.0642
     30        0.6642  0.0704
     31        0.6636  0.0718
     32        0.6631  0.0621
     33        0.6627  0.0689
     34        0.6624  0.0644
     35        0.6621  0.0668
     36   

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:775: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 762, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/skorch/classifier.py", line 357, in predict
    return (y_proba[:, 1] > self.threshold).astype('uint8')
IndexError: too many i

      3        0.9416  0.0721
      4        0.9078  0.0708
      5        0.8768  0.0628
      6        0.8486  0.0675
      7        0.8233  0.0643
      8        0.8006  0.0681
      9        0.7806  0.0685
     10        0.7629  0.0706
     11        0.7475  0.0652
     12        0.7341  0.0738
     13        0.7226  0.0653
     14        0.7128  0.0636
     15        0.7044  0.0628
     16        0.6973  0.0728
     17        0.6913  0.0628
     18        0.6862  0.0626
     19        0.6820  0.0651
     20        0.6784  0.0621
     21        0.6755  0.0658
     22        0.6730  0.0640
     23        0.6710  0.0704
     24        0.6693  0.0637
     25        0.6679  0.0800
     26        0.6667  0.0649
     27        0.6657  0.0676
     28        0.6649  0.0628
     29        0.6642  0.0660
     30        0.6637  0.0649
     31        0.6632  0.0604
     32        0.6628  0.0694
     33        0.6625  0.0605
     34        0.6622  0.0643
     35        0.6620  0.0670
     36   

In [ ]:
melhores_parametros = grid_search.best_params_
melhor_precisao = grid_search.best_score_

In [ ]:
melhores_parametros

{'batch_size': 10,
 'criterion': torch.nn.modules.loss.BCELoss,
 'max_epochs': 100,
 'module__activation': <function torch.nn.functional.relu>,
 'module__initializer': <function torch.nn.init._make_deprecate.<locals>.deprecated_init>,
 'module__neurons': 8,
 'optimizer': torch.optim.adam.Adam}

In [ ]:
melhor_precisao

0.7524153694094391